# Data: Deriving Quantities

*Purpose*: Often our data will not tell us *directly* what we want to know; in
these cases we need to *derive* new quantities from our data. In this exercise,
we'll work with `tf_mutate()` to create new columns by operating on existing
variables, and use `tf_group_by()` with `tf_summarize()` to compute aggregate
statistics (summaries!) of our data.

Aside: The data-summary verbs in grama are heavily inspired by the [dplyr](https://dplyr.tidyverse.org/) package in the R programming langauge. 

## Setup

In [ ]:
import grama as gr
DF = gr.Intention()
%matplotlib inline

# For assertion
from pandas.api.types import is_numeric_dtype

We'll be using the `diamonds` as seen in `e-vis00-basics` earlier. 

In [ ]:
from grama.data import df_diamonds

# Mutating DataFrames

A *mutation* transformation allows us to make new columns (or edit old ones) by *combining* column values. 

For example, let's remember the diamonds dataset:


In [ ]:
## NOTE: No need to edit
(
    df_diamonds
    >> gr.tf_head(5)
)

If we wanted to approximate the volume of every diamond, we could do this by multiplying their `x`, `y`, and `z` dimensions. We could create a new column with this value via `gr.tf_mutate()`:


In [ ]:
## NOTE: No need to edit
(
    df_diamonds
    >> gr.tf_mutate(volume=DF.x * DF.y * DF.z)
    >> gr.tf_head(5)
)

*Aside*: Note that we can use dot `.` notation to access columns, rather than bracket `[]` notation. This works only when the column names are valid variable names (i.e. do not contain spaces, do not contain special characters, etc.).


### __q1__ Compute a mass density

Approximate the density of the diamonds using the expression

$$\rho = \frac{\text{carat}}{xyz}.$$

Name the new column `rho`. Answer the questions under *observations* below.


In [ ]:
## TASK: Provide the approximate density as the column `rho`
df_rho = (
    df_diamonds

)

## NOTE: Use this to check your work
assert \
    "rho" in df_rho.columns, \
    "df_rho does not have a `rho` column"

assert \
    abs(df_rho.rho.median() - 6.117055e-03) < 1e-6, \
    "The values of df_rho.rho are incorrect; check your calculation"

print(
    df_rho
    >> gr.tf_filter(DF.rho < 1e6)
    >> gr.tf_select("rho")
    >> gr.tf_describe()
)


*Observations*

- What is the mean density `rho`?
  - (Your response here)
- What is the lower-quarter density `rho` (25% value)?
  - (Your response here)
- What is the upper-quarter density `rho` (75% value)?
  - (Your response here)


## Vectorized Functions

The `gr.tf_mutate()` verb carries out *vector* operations; these operations take in one or more columns, and return an entire column. For operations such as addition, this results in *elementwise* addition, as illustrated below:

| `x` | `y` | `x + y` |
|-----|-----|---------|
|  1  |  0  |    1    |
|  1  |  1  |    2    |
|  1  | -1  |    0    |


The `gr.tf_mutate()` verb accepts a wide variety of *vectorized* functions; the following table organizes these into various types of operations.

| Type | Functions |
| ---- | --------- |
| Arithmetic ops. | `DF.x + 1, DF.y - DF.x, DF.x * DF.y, DF.x / DF.y, DF.x^DF.a` |
| Modular arith. | `DF.x // DF.y` (Floor division) `DF.x % DF.y` (Remainder) |
| Logical comp. | `<, <=, >, >=, !=, ==` |
| Logarithms | `gr.log(DF.x)` |
| Offsets | `gr.lead(DF.x), gr.lag(DF.x)` |
| Cumulants | `gr.cumsum(DF.x), gr.cumprod(DF.x), gr.cummin(DF.x), gr.cummaDF.x(DF.x), gr.cummean(DF.x)` |
| Ranking | `gr.min_rank(DF.x), gr.row_number(DF.x), gr.dense_rank(DF.x), gr.percent_rank(DF.x)` |
| Data conversion | `gr.as_numeric(DF.x)`, `gr.as_str(DF.x)`, `gr.as_factor(DF.x)` |
| Control | `gr.if_else()`, `gr.case_when()` |

### __q2__ Do a type conversion

Convert the column `y` below into a numeric type.

*Hint*: Use the table above to find an appropriate data conversion function.


In [ ]:
## TASK: Convert the `y` column to a numeric type
df_converted = (
    gr.df_make(
        x=[ 1, 2, 3],
        y=["4", "5", "6"],
    )
    ## TODO: Convert `y` to numeric values

)

## NOTE: Use this to check your work
assert \
    is_numeric_dtype(df_converted.y), \
    "df_converted.y is not numeric; make sure to do a conversion"

print("Success!")

### __q3__ Sanity-check the data

The `depth` variable in `df_diamonds` is *supposedly* computed via `depth_computed = 100 * 2 * DF["z"] / (DF["x"] + DF["y"])`. Compute `diff = DF["depth"] - DF["depth_computed"]`: This is a measure of discrepancy between the given and computed depth. Answer the questions under *observations* below.

*Hint*: If you want to compute `depth_computed` and use it to compute `diff`, you'll need to use two calls of `gr.tf_mutate()`.


In [ ]:
## TASK: Compute `depth_computed` and `diff`, as described above
df_depth = (
    df_diamonds 

)

## NOTE: Use this to check your work
assert \
    abs(df_depth["diff"].median()) < 1e-14, \
    "df_depth.diff has the wrong values; check your calculation"
assert \
    abs(df_depth["diff"].mean() - 5.284249e-03) < 1e-6, \
    "df_depth.diff has the wrong values; check your calculation"

# Show the data
(
    df_depth
    >> gr.tf_select("diff")
    >> gr.tf_describe()
)


**Observation**

- What is the mean of `diff`? What is the median of `diff`?
  - (Your response here)
- What are the min and max of `diff`?
  - (Your response here)
- Remember that `diff` measures the agreement between `depth` and `depth_computed`; how well do these two quantities match?
  - (Your response here)


## Control Functions

The `gr.if_else()` and `gr.case_when()` functions are *special* vectorized functions---they allow us to use more programming-like constructs to control the output. The `gr.if_else()` helper is a simple "yes-no" function, while `gr.case_when()` is a vectorized [switch statement](https://en.wikipedia.org/wiki/Switch_statement). If you are trying to do more complicated data operations, you might want to give these functions a look.


### __q4__ Use an if/else statement

Use `gr.if_else()` to flag diamonds as `"Big"` if they have `carat >= 1` and `"Small"` otherwise.

*Hint*: Remember to read the documentation for an unfamiliar function to learn how to call it; in particular, try looking at the *Examples* section of the documentation.


In [ ]:
## TODO: Compute the `size` using `gr.if_else()`
df_size = (
    df_diamonds
    >> gr.tf_mutate(
# task-begin        
        # size=?
# task-end        

    )
)

## NOTE: Use this to check you work
assert \
    "size" in df_size.columns, \
    "df_size does not have a `size` column"

assert \
    all(df_size[df_size["size"] == "Big"].carat >= 1), \
    "Size calculation is incorrect"

df_size

# Summarizing Data Frames

The `gr.tf_summarize()` verb is used to *reduce* a DataFrame down to fewer rows using summary functions. For instance, we can compute the mean `carat` and `price` across the whole dataset of diamonds:


In [ ]:
## NOTE: No need to edit
(
    df_diamonds
    >> gr.tf_summarize(
        carat_mean=gr.mean(DF.carat),
        price_mean=gr.mean(DF.price),
    )
)

Notice that we went from having around `54,000` rows down to just one row; in this sense we have *summarized* the dataset.


## Summary functions

Summarize accepts a large number of *summary functions*; the table below lists and categorizes some of the most common summary functions.

| Type | Functions |
| ---- | --------- |
| Location | `gr.mean(DF.x), gr.median(DF.x), gr.min(DF.x), gr.max(DF.x)` |
| Spread | `gr.sd(DF.x), gr.var(DF.x), gr.IQR(DF.x)` |
| Order | `gr.first(DF.x), gr.nth(DF.x, n), gr.last(DF.x)` |
| Counts | `gr.n_distinct(DF.x)`, `gr.n()` (Total count) |
| Logical | `gr.sum(DF.x == 0)` (Count cases where `x == 0`) `gr.pr(DF.g > 0)` (Probability helper) |


### __q5__ Count the Ideal rows

Use `gr.tf_summarize()` to count the number of rows where `cut == "Ideal"`. Provide this as the column `n_ideal`.

*Hint*: The `Summary functions` table above provides a recipe for counting the number of cases where some condition is met.


In [ ]:
# TASK: Find the number of rows with cut == "Ideal"
df_nideal = (
    df_diamonds
    >> gr.tf_summarize(
        ## TODO: Compute the appropriate count
        # n_ideal=?

    )
)

## NOTE: Use this to check your work
assert \
    "n_ideal" in df_nideal.columns, \
    "df_nideal does not have a `n_ideal` column"

assert \
    df_nideal["n_ideal"][0]/23==937,\
    "The count is incorrect"

df_nideal

## Grouping by variables

The `gr.tf_summarize()` verb is helpful for getting "basic facts" about a dataset, but it is even more powerful when combined with `gr.tf_group_by()`.

The `gr.tf_group_by()` verb takes a DataFrame and "groups" it by one (or more) columns. This causes other verbs (such as `gr.tf_summarize()`) to treat the data as though it were *multiple* DataFrames; these sub-DataFrames exactly correspond to the groups determined by `gr.tf_group_by()`.

![The `gr.tf_group_by()` verb modifies the behavior of a DataFrame to treat it as a collection of multiple datasets, defined by the grouping variable(s).](images/group-by.png)

Image: Zimmerman et al.

When the input DataFrame is grouped, the `gr.tf_summarize()` verb will calculate the summary functions in a group-wise manner.

![Grouped summarize](images/grouped-summarize.png)

Image: Zimmerman et al.

Returning to the diamonds example: Let's see what adding a `gr.tf_group_by(DF.cut)` before the price and carat mean calculation does to our results.

In [ ]:
## NOTE: No need to edit
(
    df_diamonds
    >> gr.tf_group_by(DF.cut)
    >> gr.tf_summarize(
        carat_mean=gr.mean(DF.carat),
        price_mean=gr.mean(DF.price),
    )
)

Now we can compare the mean `carat` and mean `price` across different `cut` levels. Personally, I was surprised to see that the `Fair` cut diamonds have a higher mean `price` than the `Ideal` diamonds!

### __q6__ Try out a two-variable grouping

The code below groups by two variables. Uncomment the `tf_group_by()` line below, and describe how the result changes.


In [ ]:
#TASK: Uncomment the tf_group_by() below, and describe how the result changes
df_q2 = (
    df_diamonds
#    >> gr.tf_group_by(DF["color"], DF["clarity"])

    >> gr.tf_summarize(diamonds_mean=gr.mean(DF["price"]))
)

df_q2

*Observations*
- How many rows does the commented version (no grouping) return?
  - (Your response here)
- How many rows (roughly) does the uncommented version (with grouping) return?
  - (Your response here)


### __q7__ Study the color

Compute the mean `price` grouped by `color`. Answer the questions under *observations* below.


In [ ]:
# TASK: Compute the mean price, grouped by color
(
    df_diamonds
    ## TODO: Write your code here

    
    ## NOTE: We need to "ungroup" in order to re-arrange the data
    >> gr.tf_ungroup()
    >> gr.tf_arrange("price_mean")
)

*Observations*
- Which tends to be the most valuable color? Which is the least valuable?
  - (Your response here)
- The code above orders the colors according to their average price; are they in alphabetical order?
  - (Your response here)


# References

- Naupaka Zimmerman, Greg Wilson, Raniere Silva, Scott Ritchie, François Michonneau, Jeffrey Oliver, … Yuka Takemon. (2019, July). swcarpentry/r-novice-gapminder: Software Carpentry: R for Reproducible Scientific Analysis, June 2019 (Version v2019.06.1). Zenodo. http://doi.org/10.5281/zenodo.3265164